### CS4102 - Geometric Foundations of Data Analysis I
Prof. Götz Pfeiffer<br />
School of Mathematical and Statistical Sciences<br />
University of Galway

# Week 5

## 0. Questions

* How to define your own functions in python?
* How to use numpy?
* ...

## 1. Function definitions in python

* A **function** is a **named** block of code, that depends on a number of **parameters**.
* A function can be **called** by its name, with different **arguments** substituted for its parameters, as often as we like.
* By using functions, we can break up a large program into **smaller**, **easier to describe** and possibly **reusable** parts.
* Thus, using functions **reduces program complexity** and **avoids code repetition**.
* This in turn increases maintainablity of a program, and reduces oppurtunities for programming errors.
* Functions extend the programming language and increase the expressive power of a software package.

* We have already used many functions pre-defined by python and its libraries.  Such as `print`:

In [ ]:
print("Hello", end=" ")
print("World", end="")
print("!")

* To define our own function, we use a **function definition statement**.
* A function definition looks much like other **compound statements** in python.
* It consists of a **header**
    ```python
    def name_of_the_function() :
    ```
  followed by an **indented block** of code.

In [ ]:
def hello():
    print("Hello", end=" ")
    print("World", end="")
    print("!")

* The function definition statement itself has **no visible effect**.  
* Like a variable assignment, it just **assigns the name** to the block of code.
* The function can now be called and executed under its new name.

In [ ]:
hello()

* As often and whenever it is needed.

In [ ]:
hello()

* Like variable names, a function name should be **descriptive** and **meaningful**.
* It should be written as **lower case letters**, using **underscores** to separate words if necessary.

### Parameters and Arguments

* A function definition can include a number of **named parameters** (between the parentheses)

In [ ]:
def greet(name):
    print("Hello", end=" ")
    print(name, end="")
    print("!")

* When the function is called with an actual value as **argument**, this value replaces the parameter name in the function body.

In [ ]:
greet("John")
greet("Mary")

* Several parameters, if necessary are separated by commas.
* Likewise, for several arguments in the function call.

In [ ]:
def greet2(first, last):
    print("Hello", first, last, end="")
    print("!")

In [ ]:
greet2("John", "Smith")

* Omitting an actual argument for a defined parameter like here:
```python
greet2("John")
```
will cause an error.

###  Return Statement and Return Value

* The function body can include a **return statement** of the form
```python
   return value
```

In [ ]:
def sum_of_2_numbers(a, b):
    a_plus_b = a + b
    return a_plus_b

* When the function is called, this `value` will be the **return value** of the function call expression

In [ ]:
sum_of_2_numbers(12, 23)

In [ ]:
1 + sum_of_2_numbers(12, 23)

* The `value` may be itself a complex expression

In [ ]:
def product_of_2_numbers(a, b):
    return a * b

In [ ]:
product_of_2_numbers(12, 23)

* Return values are like other (constant or computed) values in python: they can be printed on the screen, stored in a variable, or used as part of another expression.

In [ ]:
val = product_of_2_numbers(17, 4)
print(val)

* If a function body does not contain a return statement, the function call will return the special value `None`, standing for ...

In [ ]:
val = greet("me")
print(val)

### Keyword Arguments

* Python provides us with some flexibility when communicating arguments to function parameters

* Suppose we have the following function:

In [ ]:
def increment(number, by):
    return number + by

* In a function call, the (different) meaning of the two arguments might no longer be obvious:

In [ ]:
increment(5, 1)

* In order to emphasize the role of `1` we can add its parameter name, turning it into a **keyword argument** (as opposed to a **positional argument**):

In [ ]:
increment(5, by=1)

* Keyword arguments must **come after** all the positional arguments.
* A function call like:
```python
increment(number=5, 1)
```
will result in an error.

### Optional Parameters and Default Arguments

* In the function definition, a parameter can be given a **default value**, turning it into an **optional parameter** (as opposed to a **required parameter**).
* Like keyword arguments, optional parameters must come after all the required parameters.

In [ ]:
def increment(number, by=1):
    return number + by

In [ ]:
increment(7)

* Of course, the default value can be overwritten by an argument, positional or keyword.

In [ ]:
increment(7, 20)

In [ ]:
increment(7, by=5)

### Variable Number of Arguments

* Some functions, like `print`, can take any number of arguments.
* To use this feature in our own function, we can decorate a parameter name with `*`:

In [ ]:
def multiply(*numbers):
    print(numbers)

* Inside the function body, this parameter will hold a **list** of argument values that can be looped over

In [ ]:
multiply(4,5,9)
multiply()

In [ ]:
def multiply(*numbers):
    for number in numbers:
        print(number)

In [ ]:
multiply(4,5,6)

In [ ]:
multiply()

In [ ]:
def multiply(*numbers):
    product = 1
    for number in numbers:
        product *= number
    return product

* Note how the final return statement is indented as part of the function body, and not as part of the for loop.

In [ ]:
multiply(4,5,9)

In [ ]:
multiply()

* This multiple arguments parameter `*args` does the opposite of list unpacking ...

In [ ]:
numbers = [2,5,4,10]
multiply(*numbers)

### Variable Keyword Arguments

* A variant of the previous allows us to pass arbitrary keyword arguments into a function call.
* For this, a parameter name is decorated with a double star `**`:

In [ ]:
def new_person(**person):
    return person    

In [ ]:
person = new_person(name="John", age=23, city="Galway")
print(person)

* Inside the function body, this parameter holds a python **dictionary** of key-value pairs (which also can be looped over):

In [ ]:
def describe_person(**person):
    for key in person:
        print(f"{key}: {person[key]}")

In [ ]:
describe_person(name="John", age=23, city="Galway")

* Such a multiple keyword parameter does the opposite of dictionary unpacking ...

In [ ]:
person = new_person(name='Mary', age=22, street='Middle Street')
describe_person(**person)

In [ ]:
person.keys()

###  Scope, Local and Global Variables

* A function provides a scope.
* Variables (and parameters) defined in a function are local to the function definition and not known or accessible from elsewhere in the code

In [ ]:
def stupid_example(param):
    number = 42 + param

* Accessing `number` or `param` after(or before) the function definition will cause an error.
* `param` is a parameter of the function, and `number` is a **local variable**.

* This means that we are free to define another function with a parameter `param`, or a local variable `number` without interfering with their use in the `stupid_example`.

* A variable defined outside any function definition is a **global variable**.
* The scope of a global variable is all of the file/notebook it is contained in.
* As a global variable's value can be accessed and modified from anywhere (within the same file/notebook), **global variables should be avoided** where possible ...
* Create functions with parameters, local variables and return values instead!

### Exercises

* ... write a function that computes a least squares best fit `B` from the data contained in `X` and `Y`.

* Write a function `fizzbuzz` that upon inputting a number, returns the string `Fizz` if
the number is divisible by $3$, the string `Buzz` if it is divisible by $5$ and `FizzBuzz` if it is divisible by both.  Otherwise it should just return the number given.

In [ ]:
def fizzbuzz(number):
    words = {3: "Fizz", 5: "Buzz"}
    text = ""
    for prime in words:
        if number % prime == 0:
            text += words[prime]
    return text if text else number

In [ ]:
fizzbuzz(15)

## 2. Numpy - a quick overview

* `numpy` is *the* library for matrix algebra in Python.
* Usually, it's name is abbreviated as `np`.

In [ ]:
import numpy as np

###  Arrays

* The fundamental data type in numpy is `ndarray`, often just called 'array',
* We can use this data type for vectors and matrice .... 

In [ ]:
np.array([1,7,3,4,9,11])  #  a vector

In [ ]:
np.array([[3,5],[-1,2]])  # a matrix

In [ ]:
np.array([[[1,2],[3,4]],[[5,6],[7,8]]]) # a 3-dim'l tensor

### Special Matrices

* There are commands for creating matrices of zeros or ones

In [ ]:
np.zeros((3,2))

In [ ]:
np.ones((3,2))

### Reshaping

* Each array has a shape: its size in each dimension.
* The shape on an array can be modified

In [ ]:
a = np.array(range(16))
a

In [ ]:
a.reshape(4,4)

In [ ]:
a.reshape(-1,8)

In [ ]:
a.reshape((2,2,2,2))

### Floating Point Ranges: `arange` vs. `linspace`

In [ ]:
np.arange(16)

In [ ]:
np.arange(1,32,2)  # start, stop (exclusive), stepsize

In [ ]:
np.arange(1, 1.3, 0.1)  # exclusve?  yes but there are rounding errors

In [ ]:
np.linspace(1, 1.3, 4)  # start, stop (inclusive), count

### Mathematical Operations

* We can use `+` and `-` for adding and subtracting matrices (of the same shape)

In [ ]:
a = np.array(range(1,7)).reshape(2,3)
a

In [ ]:
a + a

In [ ]:
a - a

* We can use `*` to multiply a matrix with a scalar ...

In [ ]:
3 * a

In [ ]:
a * 0

* ... but not for matrix multiplication :-(
* The `*` operator applied to matrices $A = (a_{ij})$ and $B= (b_{ij})$ (of the same shape) yields their *Hadamard product*:
that is the matrix $C = (c_{ij})$ (of the same shape as $A$ and $B$) with $c_{ij} = a_{ij} b_{ij}$

In [ ]:
a * a